***
# Day 9 - 머신러닝(3)

### 범주형 데이터 분석
```py
df_object = df_object.join(df_number['survived']) # survived 추가
df_object['survived'] = df_object['survived'].astype('object') 
# 데이터 타입 변경 /object 부분은 다른걸로 넣어서 결과값 도출
df_object.info().
```

freq 최빈값의 실제 발생 숫자
normalize : %으로 변경

#### 교차분석
- crosstab



***
### Data cleaning

- 결측치 처리 (결측치 = null) missing value
    - 값이 없는 것 
    - NaN: Not a Number (숫자가 아닌) ex. 연산은 하고싶은데 숫자가 아닌 문자일 때
    - Null: 아무것도 존재하지 않음
    - undefined: 정의되어 있지 않음
    - 0 : 연산이 된다. null은 아님.

- 결측치 유형 파악 (결측치 왜 발생했는지?)
    - 완전 무작위 결측 : 우연히 발생(의도한게 아닌)
        - 체중을 대답을 안하는 
    - 무작위 결측 : 특정 feature가 발생할 때도 있고 아닐 때도 있다. random하게 결측치 발생
        - 다른 feature에 따라 값이 바뀐다.
    - 비무작위 결측 : 사람의 의도대로 일부러 안넣은 것 
        - 응답하는걸 피한다.

- 결측치 탐색(확인하는 방법)
    - info()함수 사용 : 
    - isnull().sum().sum() : 전체 결측치 수
    - isnull().sum().sort_values(ascending=False) : 각 컬럼별 결측치 수
        - ascending  높은것부터 거꾸로 정렬 false
    - 정확한 해석할려면 비율로 처리★
    ```PY
    - (X_tr.isnull().sum() / X_tr.shape[0]).round(4).sort_values(ascending=False) 
      # 각 컬럼별 결측치 비율
    ```
    결측치가 발생하면 아래처럼 처리<br>
    수치형  mean / 범주형 mode

- 결측치 구간화
```py
tmp_train['age_cut'] = pd.cut(tmp_train['age'], bins=4, labels=list(range(1,5)))
# 나이를 기준으로 자른것 => 절대평가(pd.cut) / bins = 4(숫자만 바꾸면됨)
tmp_train['age_qcut'] = pd.qcut(tmp_train['age'], q=4, labels=list(range(1,5)))
# 전체 데이터를 생각하고 자른것 => 상대평가(pd.qcut) (면적을 고려해서 나눈 것)
```
- 결측치 처리
    - 결측치가 50%넘으면 제거할지말지 고려
    - 제거 / 치환 / 모델 기반 처리
    - df.dropna() : 제거
        - df.dropna(axis=0): row 삭제 (data 양이 줄어든다. 많은 데이터가 날라가서 잘 안씀)
        - df.dropna(axis=1): column 삭제 (data 양이 적게 줄어든다.)
        - df[['컬럼명']].dropna(axis=1): column 삭제
        - df.dropna(subset=['원하는 행']): 원하는 행의 결측값이 있는 부분을 기준으로 
    - df.fillna() : 치환 (ffill / bfill)
        - df[['컬럼명']].fillna(원하는 값): 원하는 값으로 채우기
        - df[['컬럼명']].fillna(method='ffill'): 결측값을 앞방향의 값으로 채운다.
        - df[['컬럼명']].fillna(method='bfill'): 결측값을 뒤방향의 값으로 채운다.
        - np.where(pd.notnull(df['null값보유컬럼']==True, df['null값보유컬럼],df['null값없는컬럼'])
    - apply()

- 불확정성 원리로 인한 오류발생 감소(데이터 훼손 down) => random화 해서 실행
    - 평균값이 올라감.
```py
X_tr['age_random'] = X_tr['age']

# random sampling
random_sampling = (X_tr['age'].dropna().sample(X_tr['age'].isnull().sum()))
# dropna() 하면, 결측치 제거 / random으로 sampling / 
# sampling개수 필요 (결측치 개수만큼 => isnull.sum()으로 결측치 알아내서 sampling)
random_sampling.index = X_tr[lambda x: x['age'].isnull()].index # index 부여
# 결측치에 대한 index값 

# NA imputation
X_tr.loc[X_tr['age'].isnull(), 'age_random'] = random_sampling
# 새로운 컬럼에 결측치 값을 주입 loc(row/col)기입해서 실행

# 확인
X_tr[['age', 'age_random']].isnull().sum()
```

-test data
train data에서 sampling (train data를 가지고 결측치를 채워야함)
```py
X_te['age_random'] = X_te['age']

# random sampling
random_sampling = (X_tr['age'].dropna().sample(X_te['age'].isnull().sum()))
random_sampling.index = X_te[lambda x: x['age'].isnull()].index # index 부여

# NA imputation
X_te.loc[X_te['age'].isnull(), 'age_random'] = random_sampling

# 확인
print(X_te[['age', 'age_random']].isnull().sum())

# 분포 시각화
fig, ax = plt.subplots(figsize=(10,6))

X_te['age'].plot(kind='kde', ax=ax, color='blue')
X_te['age_random'].plot(kind='kde', ax=ax, color='brown')

lines, labels = ax.get_legend_handles_labels()
ax.legend(lines, labels, loc='best')
```

- NA를 drop 여부 false / True
X_tr['embarked'].value_counts(dropna=False)



In [ ]:
X_tr['hasDeck'] = X_tr['deck'].isnull().apply(lambda x: 0 if x == True else 1)
# . 단위로 끊어서 보기/ null이면 true 0 / deck있으면 1 없으면 0
X_tr['hasAge'] = X_tr['age'].isnull().apply(lambda x: 0 if x == True else 1)

#### SimpleImputer (수치형)

In [ ]:
from sklearn.impute import SimpleImputer #class import

# strategy = mean, median, most_frequent
imputer = SimpleImputer(strategy="mean") #평균으로 자동으로 채워줌

X_tr['age_simple_mean'] = imputer.fit_transform(X_tr[["age"]]) 
#인스턴스/(X_tr[["age"]])의 데이터로 학습을 하고 해당결과 바탕으로 새로운 변수에 치환
#fit 학습 => train data로 실행
X_te['age_simple_mean'] = imputer.transform(X_te[["age"]])
# 학습을 안하고 채움 => train data의 학습한 데이터로 결측치를 채운다.

X_tr[['age', 'age_simple_mean']].isnull().sum()

#### 모델 기반 - KNNImputer (수치형)

In [ ]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=5)  #인스턴스 생성 /5개의 데이터만으로 결측치 채움
# 결측치 주변으로 가까운 데이터를 고려해서 값을 추출

X_tr['age_knn'] = imputer.fit_transform(X_tr[["age"]])
X_te['age_knn'] = imputer.transform(X_te[["age"]])


#### IterativeImpute  모델 기반

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

imputer = IterativeImputer(random_state=SEED)

X_tr['age_iter_none'] = imputer.fit_transform(X_tr[["age"]])
X_te['age_iter_none'] = imputer.transform(X_te[["age"]])

X_tr[['age', 'age_iter_none']].isnull().sum()

#### RandomForestRegressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor

imputer = IterativeImputer(estimator=RandomForestRegressor(verbose=0, random_state=SEED),
max_iter=10, verbose=0, imputation_order='ascending', random_state=SEED)

X_tr['age_iter_none'] = imputer.fit_transform(X_tr[["age"]])
X_te['age_iter_none'] = imputer.transform(X_te[["age"]])

X_tr[['age', 'age_iter_none']].isnull().sum()

#### 비대칭 데이터

- log함수를 쓰면 이상치도 괜찮아짐.
- 데이터가 한쪽으로 쏠린현상을 down

In [ ]:
df["fare"] = df["fare"].map(lambda i: np.log(i) if i > 0 else 0) 
# log를 이용하여 비대칭 처리
sns.displot(df["fare"], color="b", label="Skewness : %.2f"%(df["fare"].skew()))

#### 이상치 처리

In [ ]:
quantiles = df['age'].quantile([0.25, 0.75]).values

IQR = quantiles[1] - quantiles[0]
Upper_boundary = quantiles[1] + 1.5*IQR
Lower_boundary = quantiles[0] - 1.5*IQR
print('age outliers are values < {lowerboundary} or > {upperboundary}'.format(lowerboundary=Lower_boundary, upperboundary=Upper_boundary))

In [ ]:
cond1 = df['age'] > Upper_boundary
cond2 = df['age'] < Lower_boundary

cond = cond1 | cond2
outlier = df.loc[cond]
print(outlier.shape, df.shape)
outlier.head()

***
#### Feature Extraction

- 서로 독립적이면 category (상관관계 뚜렷하지 않음)
- category : object보다 작음


In [ ]:
# survived
X_tr["survived"] = X_tr["survived"].astype("int32") #사이즈를 줄임 int32
X_te["survived"] = X_te["survived"].astype("int32")

In [ ]:
X_tr["pclass"] = X_tr["pclass"].astype("category") #숫자가 아닌 형태인경우
X_te["pclass"] = X_te["pclass"].astype("category")

#### 문자열

이름, 티켓은 수치,범주형 둘다 x
호칭안에 직업,결혼유무 등 분석가능
series => map / matrix => apply

In [ ]:
# 공백제거 : strip사용 / 처리하기전에 간단하게 clening 작업
X_tr["name"] = X_tr["name"].map(lambda x: x.strip())
X_tr["ticket"] = X_tr["ticket"].map(lambda x: x.strip())

X_te["name"] = X_te["name"].map(lambda x: x.strip())
X_te["ticket"] = X_te["ticket"].map(lambda x: x.strip())

In [ ]:
dict_designation = {
    'Mr.': '남성',
    'Master.': '남성',
    'Sir.': '남성',
    'Miss.': '미혼 여성',
    'Mrs.': '기혼 여성',
    'Ms.': '미혼/기혼 여성',
    'Lady.': '숙녀',
    'Mlle.': '아가씨',
    # 직업
    'Dr.': '의사',
    'Rev.': '목사',
    'Major.': '계급',
    'Don.': '교수',
    'Col.': '군인',
    'Capt.': '군인',
    # 귀족
    'Mme.': '영부인',
    'Countess.': '백작부인',
    'Jonkheer.': '귀족'
}

def add_designation(name): # 호칭 함수
  designation = "unknown"
  for key in dict_designation.keys():
    if key in name:
      designation = key #key가 있으면 designation으로 대체한다.
      break
  return designation

X_tr['designation'] = X_tr['name'].map(lambda x: add_designation(x))
X_te['designation'] = X_te['name'].map(lambda x: add_designation(x))

X_tr.head()

***
# Feature Extraction

- grouby(), pivot으로 feature생성

In [ ]:
def sub_age(age):
  return age // 10 #10대 20대 30대식으로 feauture 추출

X_tr['sub_age'] = X_tr['age'].map(lambda x: sub_age(x))
X_te['sub_age'] = X_te['age'].map(lambda x: sub_age(x))
X_tr.head()

In [ ]:

def add_sub_embarked(row):
  return str(row['embarked']) + str(row['pclass']) + str(row['sibsp']) + str(row['parch'])

X_tr['sub_embarked'] = X_tr.apply(lambda row: add_sub_embarked(row), axis=1)
X_te['sub_embarked'] = X_te.apply(lambda row: add_sub_embarked(row), axis=1)
X_tr.head()